In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split #ok
#mostrar todas las col.
pd.set_option('display.max_columns', None) #ok
from sklearn import preprocessing #ok
from sklearn.preprocessing import StandardScaler #ok
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso #ok
from sklearn.linear_model import Ridge #ok
#Quitar los warning
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt # para dibujar
%matplotlib inline

<div class = "alert alert-success">CARGA DE DATAFRAME</div>

In [2]:
#UTILIZO DF_PRACTICA POR SI COMETO ERRORES.
df_practica = pd.read_csv('./airbnb-listings-extract.csv',
                          sep=';',
                          decimal='.')
print("Hay",df_practica.shape[0],"filas")

Hay 14780 filas


<div class = "alert alert-success">TRANSFORMACION GENERAL</div>

In [3]:
df_practica = df_practica[df_practica["City"]=="Madrid"]
print("Luego de eliminar las columnas que no pertenecen a Madrid quedaron:",df_practica.shape[0],"filas")

Luego de eliminar las columnas que no pertenecen a Madrid quedaron: 13207 filas


In [4]:
# Filtar las columnas que contengan algunas palabras clave.
#Url de perfile y id's y fechas.
filtered_cols = [col for col in df_practica.columns if any(word in col for word in ["Calendar","Time","Scrape","Pictures","Photo","URL","url","Url","ID","id","Id"])]

print("Se eliminaran las siguientes columnas antes de dividir TRAIN/TEST.")
print(filtered_cols)
df_practica = df_practica.drop(columns=filtered_cols)
print("\n quedaron:\n",df_practica.shape[1],"columnas")


Se eliminaran las siguientes columnas antes de dividir TRAIN/TEST.
['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Response Time', 'Host Thumbnail Url', 'Host Picture Url', 'Calendar Updated', 'Calendar last Scraped']

 quedaron:
 74 columnas


In [5]:
# Guardo los cambios realizados en un nuevo DF.
df_practica.to_csv("airbnb.csv", index=False)
print("Se guardó")

Se guardó


<div class = "alert alert-success">CARGA DE DATAFRAME </div>

In [6]:
full_df = pd.read_csv('./airbnb.csv', sep=',', decimal='.',index_col=None)

In [7]:
full_df = pd.read_csv('./airbnb.csv', sep=',', decimal='.')
train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=',', decimal='.', index=False)
test.to_csv('./test.csv', sep=',', decimal='.', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 

dataframe_train = pd.read_csv('./train.csv', sep=',', decimal='.')
dataframe_test = pd.read_csv('./test.csv', sep=',', decimal='.')

Dimensiones del dataset de training: (10565, 74)
Dimensiones del dataset de test: (2642, 74)


In [8]:
df_train = dataframe_train
df_train =df_train[['Price'] + [col for col in df_train.columns if col != 'Price']]

df_test = dataframe_test
df_test = df_test[['Price'] + [col for col in df_train.columns if col != 'Price']]

<div class = "alert alert-success">ANALISIS EXPLORATORIO</div>

In [9]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Host Response Rate,9225.0,94.837615,15.349749,0.000000,100.000000,100.000000,100.000000,100.000000
Host Acceptance Rate,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Host Listings Count,10563.0,9.476569,26.587255,0.000000,1.000000,2.000000,5.000000,265.000000
Host Total Listings Count,10563.0,9.476569,26.587255,0.000000,1.000000,2.000000,5.000000,265.000000
Latitude,10565.0,40.420679,0.020151,40.332908,40.410114,40.418822,40.427905,40.514247
Longitude,10565.0,-3.697294,0.023352,-3.835498,-3.707852,-3.701568,-3.693932,-3.573613
Accommodates,10565.0,3.182773,1.986991,1.000000,2.000000,2.000000,4.000000,16.000000
Bathrooms,10523.0,1.253587,0.603839,0.000000,1.000000,1.000000,1.000000,8.000000
Bedrooms,10545.0,1.293789,0.832743,0.000000,1.000000,1.000000,2.000000,10.000000
Beds,10522.0,1.981753,1.509860,1.000000,1.000000,1.000000,2.000000,16.000000


<div class = "alert alert-success">Hay 3 columnas que pueden eliminarse al no tener valores:</div>
    
    Host Acceptance Rate
    Has Availability
    Jurisdiction Names 

In [10]:
#transformacion 01
col_to_drop = ["Host Acceptance Rate","Has Availability","Jurisdiction Names"]
df_train = df_train.drop(columns=col_to_drop)

In [11]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Host Response Rate,9225.0,94.837615,15.349749,0.000000,100.000000,100.000000,100.000000,100.000000
Host Listings Count,10563.0,9.476569,26.587255,0.000000,1.000000,2.000000,5.000000,265.000000
Host Total Listings Count,10563.0,9.476569,26.587255,0.000000,1.000000,2.000000,5.000000,265.000000
Latitude,10565.0,40.420679,0.020151,40.332908,40.410114,40.418822,40.427905,40.514247
Longitude,10565.0,-3.697294,0.023352,-3.835498,-3.707852,-3.701568,-3.693932,-3.573613
Accommodates,10565.0,3.182773,1.986991,1.000000,2.000000,2.000000,4.000000,16.000000
Bathrooms,10523.0,1.253587,0.603839,0.000000,1.000000,1.000000,1.000000,8.000000
Bedrooms,10545.0,1.293789,0.832743,0.000000,1.000000,1.000000,2.000000,10.000000
Beds,10522.0,1.981753,1.509860,1.000000,1.000000,1.000000,2.000000,16.000000
Square Feet,427.0,374.084309,520.539060,0.000000,0.000000,108.000000,646.000000,2691.000000


<div class = "alert alert-success">Aqui filtro columnas que tienen un solo valor para eliminarlas.</div>

In [15]:
columns_names = list(df_train.columns) # Guardo los nombres de las columnas.
columns_filtered = df_train[columns_names].apply(lambda x: x.nunique()) # Cuento los distintos.
columns_filtered = columns_filtered[columns_filtered < 3] #filtro los distintos.
print(columns_filtered)

Experiences Offered    1
City                   1
Smart Location         2
Country Code           1
Country                1
dtype: int64


In [14]:
print("""Experiences Offered" - Todos NONE \n
"City" - Todos Madrid \n
"Smart Location" - 2 valores de Madrid \n
"Country Code" - Todos iguales \n
"Country" - Todos iguales \n
"Zipcode" - Muchos errores en los datos.""")

Experiences Offered" - Todos NONE 

"City" - Todos Madrid 

"Smart Location" - 2 valores de Madrid 

"Country Code" - Todos iguales 

"Country" - Todos iguales 

"Zipcode" - Muchos errores en los datos.


<div class = "alert alert-success">Aqui Hay un ejemplo de los errores de Zipcode.</div>

In [16]:
print("""df Zip code.value_counts())
df.shape
#Zipcode se va por tener muchos errores, quizas limpiandolos se encuentre algo de valor, para la proxima.

28012           1224
28004           1126
28005            714
28013            645
28014            411
                ... 
28051\n28051       1
2804               1
27004              1
27013              1
28054.0            1
Name: Zipcode, Length: 124, dtype: int64""")

df Zip code.value_counts())
df.shape
#Zipcode se va por tener muchos errores, quizas limpiandolos se encuentre algo de valor, para la proxima.

28012           1224
28004           1126
28005            714
28013            645
28014            411
                ... 
28051
28051       1
2804               1
27004              1
27013              1
28054.0            1
Name: Zipcode, Length: 124, dtype: int64


<div class = "alert alert-success">Borrado de las columnas antes mencionadas.</div>

In [ ]:
#transformacion 02
columns_to_drop = ["Experiences Offered", "City", "Smart Location", "Country Code", "Country","Zipcode"]
df_train = df_train.drop(columns=columns_to_drop)

<div class = "alert alert-success">Borrado de columas que no aportan valor.</div>

In [ ]:
#transformacion 04
columns_to_drop_2 = ['Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access','Interaction','House Rules',
'Host Name','Host Since','Host Location','Host About','Host Neighbourhood','Host Listings Count','Host Total Listings Count',
'Host Total Listings Count','Host Verifications','Street','State','Market',
'Latitude','Longitude','Amenities','Maximum Nights','Availability 30','Availability 60','Availability 90','Availability 365',
'First Review','Last Review','Review Scores Rating','Review Scores Accuracy','Review Scores Cleanliness','License',
'Reviews per Month','Geolocation','Features']

In [ ]:
#transformacion 05
df_train = df_train.drop(columns=columns_to_drop_2)

<div class = "alert alert-success">De estas columnas, separare las que codificare con dummies por un lado, por ej tipo de cama, y por otro lado barrios, usare mean encoder </div>

In [ ]:
#transformacion 06
#Host Response Rate - mean encoder
#Neighbourhood - mean encoder
#Neighbourhood Cleansed - mean encoder
#Property Type - mean encoder

cols= ['Neighbourhood','Neighbourhood Group Cleansed','Neighbourhood Cleansed','Property Type']

<div class = "alert alert-success">Pies a metros </div>

In [ ]:
#transformacion 07
df_train['Price'] = df_train['Price'].apply(lambda x: x * 0.3048 * 0.3048)

In [ ]:
#transformacion 08
def mean_encoding(df, cols, target):
    for col in cols:
        df.dropna(subset=[col], inplace=True)
        mean_col = df.groupby(col)[target].mean()
        df[col + '_me_en'] = df[col].map(mean_col)
    return df

df = mean_encoding(df_train, cols, 'Price')

<div class = "alert alert-success">Elimino las columnas que ya no son de utilidad, las cuales fueron reemplazadas por dummies. </div>

In [ ]:
#transformacion 09
df_train = df_train.drop(columns=cols)

<div class = "alert alert-success">Primero cambiare los nombres </div>

In [ ]:
#transformacion 10
#Primero cambiare los nombres.
new_column_names = {'Host Response Rate':'HRespRate',
                    'Room Type':'RoomType',
                    'Beds':'Beds',
                    'Bed Type':'BedType',
                    'Square Feet':'Meter',
                    'Weekly Price':'WeeklyPrice',
                    'Monthly Price':'MonthlyPrice',
                    'Security Deposit':'SecDeposit',
                    'Cleaning Fee':'CleaningFee',
                    'Guests Included':'GIncluded',
                    'Extra People':'EPeople',
                    'Minimum Nights':'MinNights',
                    'Number of Reviews':'NumReviews',
                    'Review Scores Checkin':'ScoreCheckin',
                    'Review Scores Communication':'ScoresCommunication',
                    'Review Scores Location':'ScoresLocation',
                    'Review Scores Value':'ScoresValue',
                    'Cancellation Policy':'CPolicy',
                    'Calculated host listings count':'CHostListCount',
                    'Neighbourhood_me_en':'Neighbourhood',
                    'Neighbourhood Group Cleansed_me_en':'NeighbourhoodGCleansed',
                    'Neighbourhood Cleansed_me_en':'NeighbourhoodCleansed',
                    'Property Type_me_en':'PropertyType'}

df_train.rename(columns=new_column_names, inplace=True)

<div class = "alert alert-success">Ahora debo codificar Room Type, Bed Type y Cancellation Policy. </div>

In [ ]:
#transformacion 11
df_train = pd.get_dummies(df_train, columns=['RoomType'])
df_train = pd.get_dummies(df_train, columns=['BedType'])
df_train = pd.get_dummies(df_train, columns=['CPolicy'])

<div class = "alert alert-success">Utilizo la moda para remplazar los varoles na. </div>

In [ ]:
df_train.head()

In [ ]:
#transformacion 12
mode_value = df_train.mode().iloc[0]
df_train = df_train.fillna(mode_value)

In [ ]:
for i in col_num_to_remp:
    print(i)
    #Grafico
    df_train[i].plot.hist()
    #df.boxplot(column=[i],vert=True)
    #Grafico 02
    df_train.plot(kind = 'scatter',x=i,y = 'Price')
    plt.xlabel(i)
    plt.ylabel('Price')
    plt.show()

<div class = "alert alert-success">Analizo las correlaciones.</div>

In [ ]:
corr = np.abs(df_train.drop(['Price'], axis=1).corr())

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(12, 10))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,vmin = 0.0, vmax=1.0, center=0.5,
            linewidths=.1, cmap="YlGnBu", cbar_kws={"shrink": .8})

plt.show()

In [ ]:
corr = df_train.corr()
plt.figure(figsize=(18, 15))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
df_train.corr().T

In [ ]:
df_train.dtypes

In [ ]:
<div class = "alert alert-success"> Busque outliers </div>

# Calcular el IQR
    Q1 = np.percentile(df[i].dropna(), 25)
    Q3 = np.percentile(df[i].dropna(), 75)

    IQR = Q3 - Q1
    k = 1.5
    
    # Calcular el punto de corte para los valores outliers - Tukey
    corte = Q3 + k * IQR
    print("El punto de corte optimo es:",corte)
    print("\n------------------------------\n")

<div class = "alert alert-success"> convertimos el DataFrame al formato necesario para scikit-learn </div>

In [21]:
from sklearn.feature_selection import f_regression, mutual_info_regression
data = df_train

y = data['Price'].values # nos quedamos con la 1ª columna, price
X = data.loc[:, data.columns != 'Price'].values # nos quedamos con el resto

feature_names = data.columns[data.columns != 'Price'] # nos quedamos con el resto



#valor de prueba
f_test, _ = f_regression(X, y)

#normalizarlo
f_test /= np.max(f_test)

#valor de información mutua
mi = mutual_info_regression(X, y)

#normalizarlo
mi /= np.max(mi)

# do some plotting
plt.figure(figsize=(20, 5))

plt.subplot(1,2,1)
plt.bar(range(X.shape[1]),f_test,  align="center")
plt.xticks(range(X.shape[1]),feature_names, rotation = 90)
plt.xlabel('features')
plt.ylabel('Ranking')
plt.title('$F-info$ score')

plt.subplot(1,2,2)
plt.bar(range(X.shape[1]),mi, align="center")
plt.xticks(range(X.shape[1]),feature_names, rotation = 90)
plt.xlabel('features')
plt.ylabel('Ranking')
plt.title('Mutual information score')

plt.show()

ValueError: could not convert string to float: 'Madrid Center Sunny Appartment'

In [ ]:
alpha_vector = np.logspace(-5,5,50)
param_grid = {'alpha': alpha_vector }
grid = GridSearchCV(Lasso(), scoring= 'neg_mean_squared_error', param_grid=param_grid, cv = 10,verbose=1)
grid.fit(XtrainScaled, y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

#-1 porque es negado
scores = -1*np.array(grid.cv_results_['mean_test_score'])
plt.semilogx(alpha_vector,scores,'-o')
plt.xlabel('alpha',fontsize=16)
plt.ylabel('10-Fold MSE')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(XtrainScaled,y_train)

ytrainLasso = lasso.predict(XtrainScaled)
ytestLasso  = lasso.predict(XtestScaled)
mseTrainModelLasso = mean_squared_error(y_train,ytrainLasso)
mseTestModelLasso = mean_squared_error(y_test,ytestLasso)

print('MSE Modelo Lasso (train): %0.3g' % mseTrainModelLasso)
print('MSE Modelo Lasso (test) : %0.3g' % mseTestModelLasso)

print('RMSE Modelo Lasso (train): %0.3g' % np.sqrt(mseTrainModelLasso))
print('RMSE Modelo Lasso (test) : %0.3g' % np.sqrt(mseTestModelLasso))

w = lasso.coef_
for f,wi in zip(feature_names,w):
    print(f,wi)

In [ ]:
alpha_vector = np.logspace(-5,5,50)
param_grid = {'alpha': alpha_vector }
grid = GridSearchCV(Ridge(), scoring='neg_mean_squared_error', param_grid=param_grid, cv=10, verbose=1) 
grid.fit(XtrainScaled, y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

#-1 porque es negado
scores = -1*np.array(grid.cv_results_['mean_test_score'])
plt.semilogx(alpha_vector,scores,'-o')
plt.xlabel('alpha',fontsize=16)
plt.ylabel('10-Fold MSE')
plt.show()

In [ ]:
alpha_optimo = grid.best_params_['alpha']
ridge = Ridge(alpha = alpha_optimo).fit(XtrainScaled,y_train)

ytrainRidge = ridge.predict(XtrainScaled)
ytestRidge  = ridge.predict(XtestScaled)
mseTrainModelRidge = mean_squared_error(y_train,ytrainRidge)
mseTestModelRidge = mean_squared_error(y_test,ytestRidge)

print('MSE Modelo Ridge (train): %0.3g' % mseTrainModelRidge)
print('RMSE Modelo Ridge (train): %0.3g' % np.sqrt(mseTrainModelRidge))
print('MSE Modelo Ridge (test) : %0.3g' % mseTestModelRidge)
print('RMSE Modelo Ridge (test) : %0.3g' % np.sqrt(mseTestModelRidge))

<div class = "alert alert-success">
Codigo utilizado para ver valores en columnas.
</div>

In [ ]:
#print(df_sc['Monthly Price'].value_counts())
#print(df['Monthly Price'].nunique())
#print(df['Street'])

In [ ]:
#Revisando colinealidad.
#pd.plotting.scatter_matrix(df, alpha=0.2, figsize=(20, 20), diagonal = 'hist')
#plt.show()

In [ ]:
#Aqui comienza la funcion de transformacion para Train y Test.

In [19]:
columns_to_drop = ["Host Acceptance Rate","Has Availability","Jurisdiction Names","Experiences Offered", "City", "Smart Location", "Country Code", "Country",'Zipcode',
'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access','Interaction','House Rules',
'Host Name','Host Since','Host Location','Host About','Host Neighbourhood','Host Listings Count','Host Total Listings Count',
'Host Total Listings Count','Host Verifications','Street','State','Market',
'Latitude','Longitude','Amenities','Maximum Nights','Availability 30','Availability 60','Availability 90','Availability 365',
'First Review','Last Review','Review Scores Rating','Review Scores Accuracy','Review Scores Cleanliness','License',
'Reviews per Month','Geolocation','Features']


cols= ['Neighbourhood','Neighbourhood Group Cleansed','Neighbourhood Cleansed','Property Type']


In [ ]:
new_column_names = {'Host Response Rate':'HRespRate',
                    'Room Type':'RoomType',
                    'Beds':'Beds',
                    'Bed Type':'BedType',
                    'Square Feet':'Meter',
                    'Weekly Price':'WeeklyPrice',
                    'Monthly Price':'MonthlyPrice',
                    'Security Deposit':'SecDeposit',
                    'Cleaning Fee':'CleaningFee',
                    'Guests Included':'GIncluded',
                    'Extra People':'EPeople',
                    'Minimum Nights':'MinNights',
                    'Number of Reviews':'NumReviews',
                    'Review Scores Checkin':'ScoreCheckin',
                    'Review Scores Communication':'ScoresCommunication',
                    'Review Scores Location':'ScoresLocation',
                    'Review Scores Value':'ScoresValue',
                    'Cancellation Policy':'CPolicy',
                    'Calculated host listings count':'CHostListCount',
                    'Neighbourhood_me_en':'Neighbourhood',
                    'Neighbourhood Group Cleansed_me_en':'NeighbourhoodGCleansed',
                    'Neighbourhood Cleansed_me_en':'NeighbourhoodCleansed',
                    'Property Type_me_en':'PropertyType'}




def mean_encoding(df, cols):
    for col in cols:
        df.dropna(subset=[col], inplace=True)
        mean_col = df.groupby(col)['Price'].mean()
        df[col + '_me_en'] = df[col].map(mean_col)
    return df




def transform_data(df, data_set):
    if data_set in ['train', 'test']:
        df = df.drop(columns=columns_to_drop)
        df['Price'] = df['Price'].apply(lambda x: x * 0.3048 * 0.3048)
        mean_encoding(df, cols)
        df = df.drop(columns=cols)
        df.rename(columns=new_column_names, inplace=True)
        df = pd.get_dummies(df, columns=['RoomType'])
        df = pd.get_dummies(df, columns=['BedType'])
        df = pd.get_dummies(df, columns=['CPolicy'])
        mode_value = df.mode().iloc[0]
        df = df.fillna(mode_value)
    else:
        raise ValueError('Invalid data set')
    return df
